This Notebook evaluates the study comparing the different variants of numerical fluxes for a Mach 2 supersonic flow over a wedge as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_WedgeFlow_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 4; Grid Count = 4; Path = \\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_FluxStudy }

look at all sessions

In [3]:

var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var sessions = database.Sessions;
sessions


2024-03-29 11:37:34Z

pick one session and timestep to compute the exact enthalpy

In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;

2b643461-a4fc-4470-8b84-08adf752ad39

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses=sessions.Where(s => (s.SuccessfulTermination &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
ses


#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:18 AM	2b643461...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 11:36:52 AM	8cd4658c...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:36:27 AM	031b3a93...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:34 AM	8c73340c...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [10]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M544.5,57.1 L597.9,57.1 M53.9,50.9 L75.8,54.9 L97.7,58.9 L119.7,68.2 L141.6,120.2 L163.5,140.5
 L185.4,145.0 L207.4,154.3 L229.3,159.0 L251.2,170.2 L273.1,189.2 L295.1,241.4 L317.0,355.4 L338.9,392.4
 L360.8,424.2 L382.8,465.7 L404.7,497.7 L426.6,496.9 L448.5,520.5 L470.5,517.4 L492.4,510.3 L514.3,505.6
 L536.2,499.3 L558.2,498.1 L580.1,523.2 L602.0,519.4 L623.9,512.7 L645.8,507.7 L667.8,500.7 L689.7,499.6
 L711.6,523.0 L733.5,520.1 L755.5,513.6 L777.4,508.4 L799.3,502.6 L821.2,501.4 L843.2,526.7 L865.1,521.2
 L887.0,515.5 L908.9,509.2 L930.9,503.9 L952.8,502.7 L974.7,524.4 L996.6,521.7 L1018.6,516.7 L1040.5,511.1
 L1062.4,504.3 L1084.3,498.9 L1106.3,492.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M544.5,81.1 L597.9,81.1 M53.9,50.9 L75.8,94.7 L97.7,117.7 L119.7,126.4 L141.6,126.7 L163.5,124.5
 L185.4,123.8 L207.4,123.6 L229.3,122.9 L251.2,125.7 L273.1,127.3 L295.1,133.0 L317.0,144.8 L338.9,149.9
 L360.8,190.7 L382.8,209.5 L404.7,255.3 L426.6,350.7 L448.5,437.2 L470.5,473.7 L492.4,476.4 L514.3,518.7
 L536.2,514.9 L558.2,508.5 L580.1,501.8 L602.0,494.5 L623.9,489.7 L645.8,523.6 L667.8,527.6 L689.7,526.7
 L711.6,524.7 L733.5,519.8 L755.5,514.8 L777.4,507.1 L799.3,500.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0

## Residual History

In [11]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    lts.ResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M55.7,57.1 L109.1,57.1 M53.9,55.5 L67.8,59.1 L81.7,62.6 L95.7,70.2 L109.6,88.7 L123.5,103.1
 L137.4,101.0 L151.4,106.5 L165.3,104.7 L179.2,106.4 L193.1,125.9 L207.1,176.0 L221.0,282.7 L234.9,314.6
 L248.8,341.0 L262.8,382.6 L276.7,396.7 L290.6,417.7 L304.5,407.6 L318.5,401.6 L332.4,395.5 L346.3,389.5
 L360.2,398.1 L374.2,419.1 L388.1,409.0 L402.0,403.0 L415.9,397.0 L429.8,391.0 L443.8,399.6 L457.7,420.5
 L471.6,410.5 L485.5,404.5 L499.5,398.5 L513.4,392.5 L527.3,401.1 L541.2,422.2 L555.2,411.8 L569.1,405.8
 L583.0,399.7 L596.9,393.7 L610.9,402.3 L624.8,423.4 L638.7,413.1 L652.6,407.1 L666.6,401.1 L680.5,395.1
 L694.4,389.0 L708.3,383.0 L722.3,391.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M55.7,81.1 L109.1,81.1 M53.9,47.1 L67.8,50.9 L81.7,54.2 L95.7,59.2 L109.6,65.2 L123.5,67.2
 L137.4,67.7 L151.4,71.1 L165.3,77.4 L179.2,92.3 L193.1,115.2 L207.1,165.1 L221.0,253.0 L234.9,298.9
 L248.8,327.0 L262.8,368.7 L276.7,381.5 L290.6,396.6 L304.5,390.8 L318.5,384.7 L332.4,378.7 L346.3,372.7
 L360.2,382.9 L374.2,398.1 L388.1,392.2 L402.0,386.2 L415.9,380.2 L429.8,374.1 L443.8,384.4 L457.7,399.5
 L471.6,393.7 L485.5,387.7 L499.5,381.7 L513.4,375.7 L527.3,385.9 L541.2,401.0 L555.2,395.0 L569.1,388.9
 L583.0,382.9 L596.9,376.9 L610.9,387.2 L624.8,402.2 L638.7,396.3 L652.6,390.3 L666.6,384.3 L680.5,378.2
 L694.4,372.2 L708.3,366.2 L722.3,376.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M55.7,105.1 L109.1,105.1 M53.9,55.5 L67.8,73.7 L81.7,84.4 L95.7,92.1 L109.6,95.7 L123.5,106.6
 L137.4,105.4 L151.4,106.2 L165.3,103.0 L179.2,101.0 L193.1,107.5 L207.1,105.4 L221.0,115.9 L234.9,114.4
 L248.8,145.6 L262.8,166.9 L276.7,209.1 L290.6,301.7 L304.5,367.5 L318.5,383.1 L332.4,401.8 L346.3,413.3
 L360.2,406.8 L374.2,400.3 L388.1,393.8 L402.0,402.3 L415.9,417.8 L429.8,438.0 L443.8,431.8 L457.7,425.3
 L471.6,418.7 L485.5,412.2 L499.5,405.7 L513.4,399.1 L527.3,392.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0

### Plot ShadowFields

In [12]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}